### Задача 2
---
В пункте В2 сталь поступает на условный производственный комплекс, состоящий из сталелитейного и станкостроительного заводов. Он функционирует в течение 5 лет.

Начальный запас стали составляет 3100 т (для решения; интервал для исследования — 1000–10000 т (\*)).
Исходные производственные мощности заводов соответственно:
- по стали — 4300 т в год;
- по станкам — 1300 станков в год (для решения; интервал для исследования — 100–5000 станков). (\*\*)

Сталь расходуется на производство станков (1.1 т на каждый) и собственно стали, а также на расширение производственных мощностей комплекса. При этом каждая тонна стали, направленная на ее производство, обеспечивает выпуск 3.4 т. Тонна стали, идущая на расширение производственной мощности сталелитейного завода, увеличивает последнюю на 0.1 т, а для увеличения производственных мощностей станкостроительного завода на один станок необходимо затратить 15 т стали. Реализация решения о распределении стали на следующий год осуществляется в конце очередного года планируемого периода; станкостроительный завод не может получать более половины имеющегося запаса стали.

Требуется:
1. определить план распределения стали между сталелитейным и станкостроительным заводами, обеспечивающий выпуск максимального количества станков за плановый период;
1. провести исследование (поочередно, по одному фактору) влияния начальных условий (\*) и (\*\*) на структуру и результат решения (т.е. на распределение стали и на объем выпуска станков).


In [2]:
steel_exp = variable(5, 'steel_exp') # потрачено на увеличение стали
machine_exp = variable(5, 'machine_exp') # потрачено на увеличение станков
prod_steel = variable(5, 'prod_steel') # потрачено на производство стали
prod_machine = variable(5, 'prod_machine') # потрачено на производство станков

NameError: name 'variable' is not defined

In [ ]:
problem = op(
    sum([-prod_machine[]/1.1 for i in range(0, 5)]),
    [
        # Ограничения по стали
        # первый год
        steel_exp[0] + steel_exp[0] + prod_steel[0] + prod_machine[0] == initial_steel,
        # второй год
        steel_exp[1] + machine_exp[1] + machine_exp[1] + prod_machine[1] == steel_convertion_factor * prod_steel[0],
        # третий год
        steel_exp[2] + machine_exp[2] + prod_steel[2] + prod_machine[2] == steel_convertion_factor * prod_steel[1],
        # четвертый год
        steel_exp[3] + machine_exp[3] + prod_steel[3] + prod_machine[3] == steel_convertion_factor * prod_steel[2],
        # пятый год
        steel_exp[4] + machine_exp[4] + prod_steel[4] + prod_machine[4] == steel_convertion_factor * prod_steel[3],
        
        # Ограничения на производстве стали
        steel_convertion_factor * prod_steel[0] <= steel_convertion_factor,
        steel_convertion_factor * prod_steel[1] <= initial_steel + production_rice_per_steel + ( steel_exp[0] ),
        steel_convertion_factor * prod_steel[2] <= initial_steel + production_rice_per_steel + ( steel_exp[0] + steel_exp[1] ),
        steel_convertion_factor * prod_steel[3] <= initial_steel + production_rice_per_steel + ( steel_exp[0] + steel_exp[1] + steel_exp[2] ),
        steel_convertion_factor * prod_steel[4] <= initial_steel + production_rice_per_steel + ( steel_exp[0] + steel_exp[1] + steel_exp[2] + steel_exp[3] ),
        
        # Ограничения на производстве станков
        prod_machine[0] / cost_per_machine <= machine_count,
        prod_machine[1] / cost_per_machine <= machine_count + machine_rice * ( machine_exp[0] ),
        prod_machine[2] / cost_per_machine <= machine_count + machine_rice * ( machine_exp[0] + machine_exp[1] ),
        prod_machine[3] / cost_per_machine <= machine_count + machine_rice * ( machine_exp[0] + machine_exp[1] + machine_exp[2] ),
        prod_machine[4] / cost_per_machine <= machine_count + machine_rice * ( machine_exp[0] + machine_exp[1] + machine_exp[2] + machine_exp[3] ),
        
        # Половины ограничений на производстве стали
        prod_machine[0] + machine_exp[0] <= 0.5 * ( 6700 ),
        prod_machine[1] + machine_exp[1] <= 0.5 * ( steel_convertion_factor * prod_steel[0] ),
        prod_machine[2] + machine_exp[2] <= 0.5 * ( steel_convertion_factor * prod_steel[1] ),
        prod_machine[3] + machine_exp[3] <= 0.5 * ( steel_convertion_factor * prod_steel[2] ),
        prod_machine[4] + machine_exp[4] <= 0.5 * ( steel_convertion_factor * prod_steel[3] )
    ])        

In [ ]:
problem.solve(solver = 'glpk')

In [ ]:
problem.status

In [ ]:
result = {
    'steel_exp': [steel_exp.value[i] for i in range(5)],
    'machine_exp': [machine_exp.value[i] for i in range(5)],
    'prod_steel': [prod_steel.value[i] for i in range(5)],
    'prod_machine': [prod_machine.value[i] for i in range(5)],
    'objective_value': -problem.objective.value()[0]
}    

In [ ]:
print('Results -> {}'.format(result))